In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))
import numba

# Any results you write to the current directory are saved as output.

['fork-of-preprocess-corpus', 'predict-jiaxi-2']


In [2]:
df_all = pd.read_csv('../input/predict-jiaxi-2/df_all_toeknized_filled_na_search_term.csv')

In [3]:
df_all.head()

,attribute,id,product_description,product_title,product_uid,relevance,search_term,product_description_tokens,product_title_tokens,search_term_tokens,attribute_tokens,product_description_stemmed,product_title_stemmed,search_term_stemmed,attribute_stemmed
0,Versatile connector for various 90 connections...,2,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,angle bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,angl bracket,versatil connector for variou connect and home...
1,Versatile connector for various 90 connections...,3,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,l bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,l bracket,versatil connector for variou connect and home...
2,"Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...",9,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,behr premium textured deckover is an innovativ...,behr premium textured deckover 1gal sc141 tugb...,deck over,brushrollerspray 663 in 776 in 663 in revives ...,behr premium textur deckov is an innov solid c...,behr premium textur deckov tugboat wood and co...,deck over,brushrollerspray in in in reviv wood and compo...
3,Combo Tub and Shower No Includes the trim kit ...,16,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,update your bathroom with the delta vero singl...,delta vero 1handle shower only faucet trim kit...,rain shower head,combo tub and shower no includes the trim kit ...,updat your bathroom with the delta vero single...,delta vero shower onli faucet trim kit in chro...,rain shower head,combo tub and shower no includ the trim kit on...
4,Combo Tub and Shower No Includes the trim kit ...,17,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,update your bathroom with the delta vero singl...,delta vero 1handle shower only faucet trim kit...,shower only faucet,combo tub and shower no includes the trim kit ...,updat your bathroom with the delta vero single...,delta vero shower onli faucet trim kit in chro...,shower onli faucet,combo tub and shower no includ the trim kit on...


In [4]:
df_all.isnull().sum()

attribute                       43974
id                                  0
product_description                 0
product_title                       0
product_uid                         0
relevance                      112067
search_term                         0
product_description_tokens          0
product_title_tokens                0
search_term_tokens                  0
attribute_tokens                43974
product_description_stemmed         0
product_title_stemmed               0
search_term_stemmed                 0
attribute_stemmed               43974
dtype: int64

In [5]:
df_all.fillna("",inplace=True)

In [6]:
df_all['cos_description_tfidf'] = 0
df_all['cos_title_tfidf'] = 0
df_all['cos_attribute_tfidf'] = 0

In [7]:
#df_all = df_all[:10]

In [8]:
product = df_all.drop_duplicates("product_uid")

In [9]:
#product

In [10]:
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    

def get_tfidf(text, q):
    cv = TfidfVectorizer()
    cv.fit(text)
    return cv.transform(text),cv.transform(q)


In [11]:
if False:
    total_des = product.product_description.values
    total_search = df_all.search_term.values
    total_title = product.product_title.values
    total_attr = product.product_info.values

In [12]:
product.columns

Index(['attribute', 'id', 'product_description', 'product_title',
       'product_uid', 'relevance', 'search_term', 'product_description_tokens',
       'product_title_tokens', 'search_term_tokens', 'attribute_tokens',
       'product_description_stemmed', 'product_title_stemmed',
       'search_term_stemmed', 'attribute_stemmed', 'cos_description_tfidf',
       'cos_title_tfidf', 'cos_attribute_tfidf'],
      dtype='object')

In [13]:
len(product)

106650

In [14]:
#product = product[:10]

In [15]:
total_des = product.product_description_stemmed.values
total_search = df_all.search_term_stemmed.values
total_title = product.product_title_stemmed.values
total_attr = product.attribute_stemmed.values

In [16]:
num_product = len(product)

In [17]:
term_description_matrix, des_search_mat = get_tfidf(total_des,total_search)
term_attr_matrix, attr_search_mat = get_tfidf(total_attr,total_search)
term_title_matrix, title_search_mat = get_tfidf(total_title,total_search)

In [18]:
product["des_mat"] = [i for i in term_description_matrix]
product["attr_mat"] = [i for i in term_attr_matrix]
product["title_mat"] = [i for i in term_title_matrix]

In [19]:
df_all["search_des_mat"] = [i for i in des_search_mat]
df_all["search_attr_mat"] = [i for i in attr_search_mat]
df_all["search_title_mat"] = [i for i in title_search_mat]

In [20]:
product_merge = product[["product_uid","des_mat","attr_mat","title_mat"]]

In [21]:
new_df_all = df_all.merge(product_merge,on = "product_uid",how = "outer")

In [22]:
@numba.jit
def cos(*x):
    return cosine_similarity(x[0].reshape(1,-1), x[1].reshape(1,-1))[0][0]

In [23]:
new_df_all["cos_description_tfidf"] = new_df_all[["search_des_mat","des_mat"]].apply(lambda x:cos(*x),axis = 1) 
new_df_all["cos_title_tfidf"] = new_df_all[["search_title_mat","title_mat"]].apply(lambda x:cos(*x),axis = 1) 
new_df_all["cos_attribute_tfidf"] = new_df_all[["search_attr_mat","attr_mat"]].apply(lambda x:cos(*x),axis = 1) 

In [24]:
new_df_all.head()

,attribute,id,product_description,product_title,product_uid,relevance,search_term,product_description_tokens,product_title_tokens,search_term_tokens,attribute_tokens,product_description_stemmed,product_title_stemmed,search_term_stemmed,attribute_stemmed,cos_description_tfidf,cos_title_tfidf,cos_attribute_tfidf,search_des_mat,search_attr_mat,search_title_mat,des_mat,attr_mat,title_mat
0,Versatile connector for various 90 connections...,2,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,3,angle bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,angle bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,angl bracket,versatil connector for variou connect and home...,0.181655,0.408386,0.108474,"(0, 21487)\t0.7003631580877346\n (0, 5536)\...","(0, 3516)\t0.6943144837101669\n (0, 924)\t0...","(0, 2243)\t0.6967857622566913\n (0, 632)\t0...","(0, 240987)\t0.17295095304747096\n (0, 2408...","(0, 33401)\t0.05663431949844082\n (0, 32159...","(0, 18397)\t0.5847258029435652\n (0, 17173)..."
1,Versatile connector for various 90 connections...,3,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,2.5,l bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,l bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,l bracket,versatil connector for variou connect and home...,0.000000,0.000000,0.000000,"(0, 21487)\t1.0","(0, 3516)\t1.0","(0, 2243)\t1.0","(0, 240987)\t0.17295095304747096\n (0, 2408...","(0, 33401)\t0.05663431949844082\n (0, 32159...","(0, 18397)\t0.5847258029435652\n (0, 17173)..."
2,Versatile connector for various 90 connections...,4,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,,metal l brackets,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,metal l brackets,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,metal l bracket,versatil connector for variou connect and home...,0.000000,0.000000,0.000000,"(0, 133111)\t0.5727900740586263\n (0, 21487...","(0, 18415)\t0.5281904681321918\n (0, 3516)\...","(0, 11716)\t0.6171476906484157\n (0, 2243)\...","(0, 240987)\t0.17295095304747096\n (0, 2408...","(0, 33401)\t0.05663431949844082\n (0, 32159...","(0, 18397)\t0.5847258029435652\n (0, 17173)..."
3,Versatile connector for various 90 connections...,5,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,,simpson sku able,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,simpson sku able,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,simpson sku abl,versatil connector for variou connect and home...,0.068423,0.577868,0.100501,"(0, 193901)\t0.6569067547071865\n (0, 19258...","(0, 27142)\t0.6383309638889721\n (0, 26935)...","(0, 17173)\t1.0","(0, 240987)\t0.17295095304747096\n (0, 2408...","(0, 33401)\t0.05663431949844082\n (0, 32159...","(0, 18397)\t0.5847258029435652\n (0, 17173)..."
4,Versatile connector for various 90 connections...,6,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,,simpson strong ties,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,simpson strong ties,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,simpson strong tie,versatil connector for variou connect and home...,0.079600,0.267189,0.187175,"(0, 215757)\t0.6051441730547947\n (0, 20581...","(0, 30218)\t0.6266301988700662\n (0, 28852)...","(0, 19282)\t0.5332915599681283\n (0, 18391)...","(0, 240987)\t0.17295095304747096\n (

In [25]:
new_df_all[["cos_description_tfidf","cos_title_tfidf","cos_attribute_tfidf"]].to_csv("df_all_tfidf.csv",index = False)

In [26]:
new_df_all.to_csv("df_all_tfidf_with_matrix.csv",index = False)